# データ整形：TransRec

In [1]:
import pandas as pd
import scipy.sparse as sp
import numpy as np
from collections import defaultdict
import copy
import os

In [2]:
# config
dataset_name = 'MovieLens100k'
filename        = '../../data/' + dataset_name + '/ratings.csv'

df = pd.read_csv(filename, sep=',', header=None,
                names=['user_id', 'item_id', 'rating', 'time'], index_col=False)

In [3]:
df

,user_id,item_id,rating,time
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [6]:
item_list = df['item_id']
item_list

0           31
1         1029
2         1061
3         1129
4         1172
5         1263
6         1287
7         1293
8         1339
9         1343
10        1371
11        1405
12        1953
13        2105
14        2150
15        2193
16        2294
17        2455
18        2968
19        3671
20          10
21          17
22          39
23          47
24          50
25          52
26          62
27         110
28         144
29         150
          ... 
99974     4034
99975     4306
99976     4308
99977     4880
99978     4886
99979     4896
99980     4963
99981     4973
99982     4993
99983     4995
99984     5010
99985     5218
99986     5299
99987     5349
99988     5377
99989     5445
99990     5464
99991     5669
99992     5816
99993     5902
99994     5952
99995     5989
99996     5991
99997     5995
99998     6212
99999     6268
100000    6269
100001    6365
100002    6385
100003    6565
Name: item_id, Length: 100004, dtype: int64

In [7]:
print(74 in df['item_id'])

True


In [8]:
item_list

0           31
1         1029
2         1061
3         1129
4         1172
5         1263
6         1287
7         1293
8         1339
9         1343
10        1371
11        1405
12        1953
13        2105
14        2150
15        2193
16        2294
17        2455
18        2968
19        3671
20          10
21          17
22          39
23          47
24          50
25          52
26          62
27         110
28         144
29         150
          ... 
99974     4034
99975     4306
99976     4308
99977     4880
99978     4886
99979     4896
99980     4963
99981     4973
99982     4993
99983     4995
99984     5010
99985     5218
99986     5299
99987     5349
99988     5377
99989     5445
99990     5464
99991     5669
99992     5816
99993     5902
99994     5952
99995     5989
99996     5991
99997     5995
99998     6212
99999     6268
100000    6269
100001    6365
100002    6385
100003    6565
Name: item_id, Length: 100004, dtype: int64

In [9]:
item_list = item_list.sort_values()

In [10]:
item_list = item_list.reset_index(drop=True)

In [11]:
item_list = item_list.unique()

In [14]:
print(75 in item_list)

False


In [15]:
for item in item_list:
    print(item)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
52
53
54
55
57
58
59
60
61
62
63
64
65
66
68
69
70
71
72
73
74
76
77
78
79
80
81
82
83
84
85
86
87
88
89
92
93
94
95
96
97
98
99
100
101
102
103
104
105
107
108
110
111
112
113
114
116
117
118
119
121
122
123
124
125
126
129
130
131
132
135
137
140
141
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
183
184
185
186
187
188
189
190
191
193
194
195
196
198
199
200
201
202
203
204
205
206
207
208
209
211
213
214
215
216
217
218
219
220
222
223
224
225
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
285
287
288
289
290
291
292
293
294
295
296
299
300
301
302
303
304
305
306
3

3531
3534
3535
3536
3537
3538
3539
3540
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3559
3560
3563
3564
3565
3566
3567
3568
3569
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3584
3587
3588
3590
3591
3593
3594
3596
3598
3599
3600
3602
3604
3605
3606
3608
3609
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3626
3627
3628
3629
3632
3633
3634
3635
3637
3638
3639
3640
3643
3646
3649
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3668
3669
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3751
3752
3753
3754
3755
3757
3758
3759
3760
3761
3763
3764
3765
3766
3768
3769
3770
3771
3773
3774
3775


42191
42197
42351
42418
42632
42677
42681
42718
42721
42723
42725
42728
42730
42732
42734
42738
42740
42783
42900
42935
42946
42958
43177
43267
43333
43351
43376
43391
43396
43419
43556
43558
43560
43635
43679
43708
43744
43832
43836
43838
43869
43871
43899
43904
43908
43910
43917
43919
43921
43928
43930
43932
43936
44004
44022
44073
44191
44193
44195
44197
44199
44204
44225
44245
44301
44317
44397
44421
44511
44555
44587
44597
44613
44633
44665
44671
44694
44709
44717
44731
44759
44761
44788
44828
44840
44849
44864
44911
44972
44974
45000
45028
45062
45081
45172
45179
45183
45186
45208
45210
45221
45382
45431
45440
45442
45447
45499
45501
45506
45517
45521
45662
45666
45668
45672
45679
45720
45722
45726
45728
45730
45732
45837
45880
45928
45942
45950
45969
45987
46062
46322
46335
46337
46347
46530
46544
46559
46561
46574
46578
46664
46723
46772
46850
46855
46865
46919
46948
46965
46967
46970
46972
46974
46976
47044
47092
47099
47122
47124
47148
47152
47196
47200
47202
47254
47261
4727

161944
162376
162542
162672
163949


In [16]:
item_list.max()

163949

In [17]:
len(item_list)

9066

In [18]:
item_list[9065]

163949

In [19]:
item_idex_master = {}

In [29]:
for i in range(len(item_list)):
    item_idex_master[ item_list[i]] = i

i = 0
1
i = 1
2
i = 2
3
i = 3
4
i = 4
5
i = 5
6
i = 6
7
i = 7
8
i = 8
9
i = 9
10
i = 10
11
i = 11
12
i = 12
13
i = 13
14
i = 14
15
i = 15
16
i = 16
17
i = 17
18
i = 18
19
i = 19
20
i = 20
21
i = 21
22
i = 22
23
i = 23
24
i = 24
25
i = 25
26
i = 26
27
i = 27
28
i = 28
29
i = 29
30
i = 30
31
i = 31
32
i = 32
34
i = 33
35
i = 34
36
i = 35
37
i = 36
38
i = 37
39
i = 38
40
i = 39
41
i = 40
42
i = 41
43
i = 42
44
i = 43
45
i = 44
46
i = 45
47
i = 46
48
i = 47
49
i = 48
50
i = 49
52
i = 50
53
i = 51
54
i = 52
55
i = 53
57
i = 54
58
i = 55
59
i = 56
60
i = 57
61
i = 58
62
i = 59
63
i = 60
64
i = 61
65
i = 62
66
i = 63
68
i = 64
69
i = 65
70
i = 66
71
i = 67
72
i = 68
73
i = 69
74
i = 70
76
i = 71
77
i = 72
78
i = 73
79
i = 74
80
i = 75
81
i = 76
82
i = 77
83
i = 78
84
i = 79
85
i = 80
86
i = 81
87
i = 82
88
i = 83
89
i = 84
92
i = 85
93
i = 86
94
i = 87
95
i = 88
96
i = 89
97
i = 90
98
i = 91
99
i = 92
100
i = 93
101
i = 94
102
i = 95
103
i = 96
104
i = 97
105
i = 98
107
i = 99
108
i = 100
110

i = 1511
1949
i = 1512
1950
i = 1513
1951
i = 1514
1952
i = 1515
1953
i = 1516
1954
i = 1517
1955
i = 1518
1956
i = 1519
1957
i = 1520
1958
i = 1521
1959
i = 1522
1960
i = 1523
1961
i = 1524
1962
i = 1525
1963
i = 1526
1964
i = 1527
1965
i = 1528
1966
i = 1529
1967
i = 1530
1968
i = 1531
1969
i = 1532
1970
i = 1533
1971
i = 1534
1972
i = 1535
1973
i = 1536
1974
i = 1537
1975
i = 1538
1976
i = 1539
1977
i = 1540
1978
i = 1541
1979
i = 1542
1980
i = 1543
1981
i = 1544
1982
i = 1545
1983
i = 1546
1984
i = 1547
1985
i = 1548
1986
i = 1549
1987
i = 1550
1988
i = 1551
1989
i = 1552
1990
i = 1553
1991
i = 1554
1992
i = 1555
1993
i = 1556
1994
i = 1557
1995
i = 1558
1996
i = 1559
1997
i = 1560
1998
i = 1561
1999
i = 1562
2000
i = 1563
2001
i = 1564
2002
i = 1565
2003
i = 1566
2004
i = 1567
2005
i = 1568
2006
i = 1569
2007
i = 1570
2008
i = 1571
2009
i = 1572
2010
i = 1573
2011
i = 1574
2012
i = 1575
2013
i = 1576
2014
i = 1577
2015
i = 1578
2016
i = 1579
2017
i = 1580
2018
i = 1581
2019
i = 15

i = 2417
3008
i = 2418
3010
i = 2419
3011
i = 2420
3013
i = 2421
3015
i = 2422
3016
i = 2423
3017
i = 2424
3018
i = 2425
3019
i = 2426
3020
i = 2427
3021
i = 2428
3022
i = 2429
3024
i = 2430
3025
i = 2431
3028
i = 2432
3029
i = 2433
3030
i = 2434
3031
i = 2435
3032
i = 2436
3033
i = 2437
3034
i = 2438
3035
i = 2439
3036
i = 2440
3037
i = 2441
3038
i = 2442
3039
i = 2443
3040
i = 2444
3041
i = 2445
3042
i = 2446
3043
i = 2447
3044
i = 2448
3045
i = 2449
3046
i = 2450
3047
i = 2451
3048
i = 2452
3051
i = 2453
3052
i = 2454
3053
i = 2455
3054
i = 2456
3055
i = 2457
3057
i = 2458
3060
i = 2459
3061
i = 2460
3062
i = 2461
3063
i = 2462
3064
i = 2463
3066
i = 2464
3067
i = 2465
3068
i = 2466
3069
i = 2467
3070
i = 2468
3071
i = 2469
3072
i = 2470
3074
i = 2471
3075
i = 2472
3076
i = 2473
3077
i = 2474
3078
i = 2475
3079
i = 2476
3081
i = 2477
3082
i = 2478
3083
i = 2479
3086
i = 2480
3087
i = 2481
3088
i = 2482
3089
i = 2483
3090
i = 2484
3091
i = 2485
3093
i = 2486
3094
i = 2487
3095
i = 24

i = 3550
4501
i = 3551
4502
i = 3552
4503
i = 3553
4506
i = 3554
4508
i = 3555
4509
i = 3556
4511
i = 3557
4515
i = 3558
4516
i = 3559
4518
i = 3560
4519
i = 3561
4520
i = 3562
4521
i = 3563
4522
i = 3564
4523
i = 3565
4524
i = 3566
4526
i = 3567
4527
i = 3568
4528
i = 3569
4529
i = 3570
4531
i = 3571
4532
i = 3572
4533
i = 3573
4534
i = 3574
4535
i = 3575
4537
i = 3576
4538
i = 3577
4539
i = 3578
4541
i = 3579
4543
i = 3580
4544
i = 3581
4545
i = 3582
4546
i = 3583
4550
i = 3584
4552
i = 3585
4553
i = 3586
4555
i = 3587
4557
i = 3588
4558
i = 3589
4559
i = 3590
4560
i = 3591
4561
i = 3592
4562
i = 3593
4563
i = 3594
4564
i = 3595
4565
i = 3596
4566
i = 3597
4567
i = 3598
4568
i = 3599
4569
i = 3600
4570
i = 3601
4571
i = 3602
4572
i = 3603
4573
i = 3604
4577
i = 3605
4578
i = 3606
4584
i = 3607
4585
i = 3608
4587
i = 3609
4589
i = 3610
4590
i = 3611
4591
i = 3612
4592
i = 3613
4593
i = 3614
4600
i = 3615
4602
i = 3616
4603
i = 3617
4605
i = 3618
4606
i = 3619
4608
i = 3620
4610
i = 36

6638
i = 4730
6639
i = 4731
6641
i = 4732
6643
i = 4733
6644
i = 4734
6645
i = 4735
6646
i = 4736
6650
i = 4737
6658
i = 4738
6659
i = 4739
6662
i = 4740
6663
i = 4741
6664
i = 4742
6665
i = 4743
6666
i = 4744
6669
i = 4745
6671
i = 4746
6676
i = 4747
6678
i = 4748
6679
i = 4749
6684
i = 4750
6686
i = 4751
6690
i = 4752
6695
i = 4753
6703
i = 4754
6706
i = 4755
6707
i = 4756
6708
i = 4757
6709
i = 4758
6710
i = 4759
6711
i = 4760
6713
i = 4761
6718
i = 4762
6721
i = 4763
6722
i = 4764
6723
i = 4765
6724
i = 4766
6725
i = 4767
6728
i = 4768
6730
i = 4769
6731
i = 4770
6732
i = 4771
6735
i = 4772
6739
i = 4773
6743
i = 4774
6744
i = 4775
6748
i = 4776
6751
i = 4777
6753
i = 4778
6754
i = 4779
6755
i = 4780
6757
i = 4781
6760
i = 4782
6762
i = 4783
6763
i = 4784
6764
i = 4785
6765
i = 4786
6768
i = 4787
6769
i = 4788
6770
i = 4789
6771
i = 4790
6772
i = 4791
6773
i = 4792
6774
i = 4793
6776
i = 4794
6777
i = 4795
6779
i = 4796
6780
i = 4797
6782
i = 4798
6783
i = 4799
6785
i = 4800
6786
i

i = 6260
39941
i = 6261
40148
i = 6262
40226
i = 6263
40278
i = 6264
40339
i = 6265
40412
i = 6266
40414
i = 6267
40491
i = 6268
40574
i = 6269
40581
i = 6270
40583
i = 6271
40597
i = 6272
40614
i = 6273
40629
i = 6274
40723
i = 6275
40732
i = 6276
40815
i = 6277
40817
i = 6278
40819
i = 6279
40826
i = 6280
40833
i = 6281
40851
i = 6282
40870
i = 6283
40946
i = 6284
40959
i = 6285
41226
i = 6286
41285
i = 6287
41336
i = 6288
41527
i = 6289
41566
i = 6290
41569
i = 6291
41571
i = 6292
41573
i = 6293
41714
i = 6294
41716
i = 6295
41769
i = 6296
41863
i = 6297
41912
i = 6298
41997
i = 6299
42002
i = 6300
42004
i = 6301
42007
i = 6302
42009
i = 6303
42011
i = 6304
42013
i = 6305
42015
i = 6306
42163
i = 6307
42191
i = 6308
42197
i = 6309
42351
i = 6310
42418
i = 6311
42632
i = 6312
42677
i = 6313
42681
i = 6314
42718
i = 6315
42721
i = 6316
42723
i = 6317
42725
i = 6318
42728
i = 6319
42730
i = 6320
42732
i = 6321
42734
i = 6322
42738
i = 6323
42740
i = 6324
42783
i = 6325
42900
i = 6326
4

i = 7689
83359
i = 7690
83361
i = 7691
83374
i = 7692
83411
i = 7693
83480
i = 7694
83506
i = 7695
83603
i = 7696
83613
i = 7697
83803
i = 7698
83827
i = 7699
83829
i = 7700
83910
i = 7701
83976
i = 7702
84098
i = 7703
84116
i = 7704
84152
i = 7705
84154
i = 7706
84160
i = 7707
84187
i = 7708
84236
i = 7709
84304
i = 7710
84312
i = 7711
84374
i = 7712
84392
i = 7713
84395
i = 7714
84414
i = 7715
84506
i = 7716
84601
i = 7717
84615
i = 7718
84637
i = 7719
84696
i = 7720
84772
i = 7721
84844
i = 7722
84847
i = 7723
84944
i = 7724
84950
i = 7725
84952
i = 7726
84954
i = 7727
85016
i = 7728
85020
i = 7729
85022
i = 7730
85025
i = 7731
85056
i = 7732
85131
i = 7733
85179
i = 7734
85213
i = 7735
85261
i = 7736
85316
i = 7737
85342
i = 7738
85367
i = 7739
85394
i = 7740
85397
i = 7741
85399
i = 7742
85401
i = 7743
85412
i = 7744
85414
i = 7745
85438
i = 7746
85510
i = 7747
85572
i = 7748
85612
i = 7749
85736
i = 7750
85774
i = 7751
85780
i = 7752
85788
i = 7753
85796
i = 7754
85881
i = 7755
8

i = 8779
127319
i = 8780
127728
i = 8781
128360
i = 8782
128512
i = 8783
128520
i = 8784
128592
i = 8785
128606
i = 8786
128616
i = 8787
128620
i = 8788
128846
i = 8789
129009
i = 8790
129191
i = 8791
129250
i = 8792
129313
i = 8793
129354
i = 8794
129364
i = 8795
129428
i = 8796
129514
i = 8797
129653
i = 8798
129657
i = 8799
129659
i = 8800
129737
i = 8801
129937
i = 8802
130073
i = 8803
130083
i = 8804
130087
i = 8805
130351
i = 8806
130448
i = 8807
130450
i = 8808
130452
i = 8809
130490
i = 8810
130520
i = 8811
130522
i = 8812
130576
i = 8813
130580
i = 8814
130628
i = 8815
130634
i = 8816
130642
i = 8817
130682
i = 8818
130960
i = 8819
130970
i = 8820
130980
i = 8821
131013
i = 8822
131168
i = 8823
131451
i = 8824
131714
i = 8825
131724
i = 8826
131830
i = 8827
132046
i = 8828
132074
i = 8829
132146
i = 8830
132157
i = 8831
132333
i = 8832
132462
i = 8833
132480
i = 8834
132488
i = 8835
132496
i = 8836
132618
i = 8837
132796
i = 8838
132888
i = 8839
132952
i = 8840
132961
i = 8841

In [30]:
item_idex_master

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 34: 32,
 35: 33,
 36: 34,
 37: 35,
 38: 36,
 39: 37,
 40: 38,
 41: 39,
 42: 40,
 43: 41,
 44: 42,
 45: 43,
 46: 44,
 47: 45,
 48: 46,
 49: 47,
 50: 48,
 52: 49,
 53: 50,
 54: 51,
 55: 52,
 57: 53,
 58: 54,
 59: 55,
 60: 56,
 61: 57,
 62: 58,
 63: 59,
 64: 60,
 65: 61,
 66: 62,
 68: 63,
 69: 64,
 70: 65,
 71: 66,
 72: 67,
 73: 68,
 74: 69,
 76: 70,
 77: 71,
 78: 72,
 79: 73,
 80: 74,
 81: 75,
 82: 76,
 83: 77,
 84: 78,
 85: 79,
 86: 80,
 87: 81,
 88: 82,
 89: 83,
 92: 84,
 93: 85,
 94: 86,
 95: 87,
 96: 88,
 97: 89,
 98: 90,
 99: 91,
 100: 92,
 101: 93,
 102: 94,
 103: 95,
 104: 96,
 105: 97,
 107: 98,
 108: 99,
 110: 100,
 111: 101,
 112: 102,
 113: 103,
 114: 104,
 116: 105,
 117: 106,
 118: 107,
 119: 108,
 121: 109,
 

In [31]:
item_id= list(item_idex_master.keys())

In [32]:
item_id

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 108,
 110,
 111,
 112,
 113,
 114,
 116,
 117,
 118,
 119,
 121,
 122,
 123,
 124,
 125,
 126,
 129,
 130,
 131,
 132,
 135,
 137,
 140,
 141,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 193,
 194,
 195,
 196,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 20

In [33]:
fixed_item_idx =  list(item_idex_master.values())

In [34]:
fixed_item_idx

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [35]:
item_ids = df['item_id']
fixed_item_ids = []

In [36]:
for item_id in item_ids:
    fixed_item_ids.append(item_idex_master[item_id])

In [37]:
fixed_item_ids

[30,
 833,
 859,
 906,
 931,
 1017,
 1041,
 1047,
 1083,
 1087,
 1111,
 1140,
 1515,
 1665,
 1708,
 1743,
 1815,
 1962,
 2380,
 2925,
 9,
 16,
 37,
 45,
 48,
 49,
 58,
 100,
 123,
 129,
 132,
 140,
 144,
 147,
 163,
 164,
 184,
 195,
 196,
 198,
 207,
 220,
 225,
 233,
 237,
 238,
 244,
 245,
 262,
 266,
 268,
 280,
 283,
 285,
 304,
 314,
 315,
 321,
 322,
 328,
 331,
 334,
 335,
 336,
 341,
 346,
 357,
 361,
 403,
 406,
 416,
 422,
 427,
 432,
 444,
 447,
 454,
 455,
 461,
 472,
 481,
 483,
 493,
 494,
 495,
 518,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 543,
 567,
 607,
 56,
 100,
 219,
 239,
 266,
 284,
 320,
 321,
 341,
 472,
 521,
 524,
 525,
 527,
 617,
 642,
 699,
 954,
 966,
 990,
 1025,
 1118,
 1253,
 1359,
 1455,
 1590,
 1834,
 2010,
 2156,
 2162,
 2173,
 2212,
 2273,
 2288,
 2374,
 2599,
 2804,
 3157,
 4081,
 4255,
 4604,
 5017,
 5116,
 5467,
 5472,
 5888,
 6363,
 6536,
 6579,
 6892,
 7708,
 9,
 32,
 102,
 122,
 132,
 152,
 163,
 232,
 259,
 266,
 294,
 314,
 321,
 322,
 

In [38]:
df['item_id'] = fixed_item_ids

In [39]:
df

,user_id,item_id,rating,time
0,1,30,2.5,1260759144
1,1,833,3.0,1260759179
2,1,859,3.0,1260759182
3,1,906,2.0,1260759185
4,1,931,4.0,1260759205
5,1,1017,2.0,1260759151
6,1,1041,2.0,1260759187
7,1,1047,2.0,1260759148
8,1,1083,3.5,1260759125
9,1,1087,2.0,1260759131


In [40]:
max(df['item_id'])

9065

In [ ]:
print('Dataset size: {}'.format(len(df)))

以下のデータ数でサンプルを作成する（以下はそれぞれの数字／実際はこれの倍の大きさ）   

- MovieLens20M
    - Train: 14,000,000    
    - Eval: 4,000,000    
    - Test: 2,000,263
- MovieLens100k
    - Train: 70,000
    - Eval: 20,000
    - TestL 10,000

In [ ]:
train_df = df.iloc[:70000,:]
varidation_df = df.iloc[70000:90000,:]
test_df = df.iloc[90000:,:]

In [ ]:
train_df.head()

In [ ]:
varidation_df.head()

In [ ]:
test_df.head()

In [ ]:
user_min=5 
item_min=5

In [ ]:
print('\tnum_users = ' + str(len(df['user_id'].unique())))
print('\tnum_items = ' + str(len(df['item_id'].unique())))
print('\tdf_shape  = ' + str(df.shape))

In [ ]:
user_counts = df['user_id'].value_counts()
print('Collected user counts...')
item_counts = df['item_id'].value_counts()
print('Collected item counts...')

In [ ]:
# Filter based on user and item counts
df = df[df.apply(
        lambda x: user_counts[x['user_id']] >= user_min, axis=1)]
print('User filtering done...')

df = df[df.apply(
        lambda x: item_counts[x['item_id']] >= item_min, axis=1)]
print('Item filtering done...')

print('Second pass')
print('\tnum_users = ' + str(len(df['user_id'].unique())))
print('\tnum_items = ' + str(len(df['item_id'].unique())))
print('\tdf_shape  = ' + str(df.shape))

In [ ]:
# Normalize temporal values
print('Normalizing temporal values...')
mean = df['time'].mean()
std  = df['time'].std()
ONE_YEAR = (60 * 60 * 24 * 365) / mean
ONE_DAY  = (60 * 60 * 24) / mean
df['time'] = (df['time'] - mean) / std

In [ ]:
# 時系列にソート
df = df.sort_values('time')
# indexを振り直す
df = df.reset_index(drop=True)

## dataを整形

In [ ]:
def shaping_data_set(df):
        # 時系列にソート
        df = df.sort_values('time')
        # indexを振り直す
        df = df.reset_index(drop=True)
        
        users = df['user_id'].unique()
        users_list = df['user_id']
        item_list = df['item_id']

        target_data_set = {}

        for target_user in users:
            print('-------------------------------------')
            print(target_user)
            target_users_item_list = []
            for user, item in zip(users_list, item_list):
                if user == target_user:
                    print(item)
                    target_users_item_list.append(item)
            target_data_set[target_user] = target_users_item_list
        print(target_users_item_list)
        print('-------------------------------------')


        return target_data_set

In [ ]:
user_train =  shaping_data_set(train_df)

In [ ]:
user_train

In [ ]:
user_validation = shaping_data_set(varidation_df)

In [ ]:
user_test = shaping_data_set(test_df)

In [ ]:
# 整形したdictをファイル出力
np.save('user_train', user_train)
np.save('user_validation', user_validation)
np.save('user_test', user_test)